In [1]:
config = open('ubuntu_config').read()

In [32]:
import re
starter = {}
ender = {}
for m in re.finditer(r'#\n# (.*?)\n#\n', config):
    starter[m.group(1)] = m
for m in re.finditer(r'# end of (.*?)\n', config):
    ender[m.group(1)] = m

In [100]:
[c for c in starter if c not in ender]

['CPU frequency scaling drivers',
 'shared options',
 'Xtables combined modules',
 'Xtables targets',
 'Xtables matches',
 'IPVS transport protocol load balancing support',
 'IPVS scheduler',
 'IPVS SH scheduler',
 'IPVS MH scheduler',
 'IPVS application helper',
 'Queueing/Scheduling',
 'Classification',
 'Packet Radio protocols',
 'PC-card bridges',
 'User Modules And Translation Layers',
 'Note that in some cases UBI block is preferred. See MTD_UBI_BLOCK.',
 'Disk-On-Chip Device Drivers',
 'Raw/parallel NAND flash controllers',
 'Misc',
 'Protocols',
 'Parallel IDE high-level drivers',
 'Parallel IDE protocol modules',
 'SCSI support type (disk, tape, CD-ROM)',
 'Controllers with non-SFF native interface',
 'SFF controllers with custom DMA interface',
 'SATA SFF controllers with BMDMA',
 'PATA SFF controllers with BMDMA',
 'PIO-only SFF controllers',
 'Generic fallback / legacy drivers',
 'MII PHY device drivers',
 'MCTP Device Drivers',
 'MDIO Multiplexers',
 'mISDN hardware driver

In [168]:
locs = [("Start: " + c, m.start(), m.end()) for c, m in starter.items()] \
    + [("End: " + c, m.start(), m.end()) for c, m in ender.items()]
locs.sort(key=lambda x: x[1])

groups_match = [(c, starter[c], ender.get(c, None)) for c in starter.keys()]
groups_match.sort(key=lambda x: x[1].start())
_group_ordered = []
# for n, m1, m2 in groups_match:
#     if len(_group_ordered) > 0 and _group_ordered[-1][2] == -1:
#         _group_ordered[-1] = (_group_ordered[-1][0], _group_ordered[-1][1], m1.start())
#     _group_ordered.append((n, m1.start(), m2.end() if m2 else -1))
for n, m1, m2 in groups_match:
    start = m1.start()
    end = m2.end() if m2 else -1
    if not m2:
        for loc in locs:
            if loc[1] > start:
                end = loc[1]
                break
    _group_ordered.append((n, start, end))
assert all(c[2] != -1 for c in _group_ordered)

In [169]:
new_group_ordered = []
last_group = None
for group in _group_ordered:
    last_group = None
    for g in locs:
        if group[1] - g[2] < 0: # closest ending
            break
        last_group = g
    if last_group and config[last_group[2]:group[1]].strip():
        new_group_ordered.append(("After-" + last_group[0], last_group[2], group[1]))
    new_group_ordered.append(group)
    
_group_ordered = new_group_ordered

In [170]:
for c in _group_ordered:
    print(c)

('General setup', 576, 6747)
('After-Start: General setup', 596, 1482)
('IRQ subsystem', 1482, 2028)
('After-End: IRQ subsystem', 2028, 2377)
('Timers subsystem', 2377, 2593)
('After-End: Timers subsystem', 2593, 2666)
('BPF subsystem', 2666, 2908)
('After-End: BPF subsystem', 2908, 3018)
('CPU/Task time and stats accounting', 3018, 3417)
('After-End: CPU/Task time and stats accounting', 3417, 3442)
('RCU Subsystem', 3442, 3697)
('After-End: RCU Subsystem', 3697, 3928)
('Scheduler features', 3928, 4032)
('After-End: Scheduler features', 4032, 6202)
('Kernel Performance Events And Counters', 6202, 6359)
('After-End: General setup', 6747, 7727)
('Processor type and features', 7727, 12341)
('After-Start: Processor type and features', 7761, 9829)
('Performance monitoring', 9829, 10064)
('After-End: Processor type and features', 12341, 12446)
('Power management and ACPI options', 12446, 16179)
('After-Start: Power management and ACPI options', 12486, 15064)
('CPU Frequency scaling', 15064, 

In [171]:
from collections import OrderedDict
group_ordered = list(_group_ordered)
g_groupdict = OrderedDict()
cur_group = None
def processGroup(groupdict, cur_group):
    groupdict[cur_group] = OrderedDict()
    changed = True
    while changed:
        changed = False
        for i, group in enumerate(group_ordered):
            s = group[1]
            e = group[2]
            if s > cur_group[1] and s < cur_group[2]:
                childGroup = group_ordered.pop(i)
                processGroup(groupdict[cur_group], childGroup)
                changed = True
                break
            if not (e > cur_group[1] and s < cur_group[2]):
                break

processGroup(g_groupdict, ('root', 0, 0xffffffffffff))
g_groupdict = g_groupdict[next(iter(g_groupdict))]
assert not group_ordered

In [172]:
def printGroups(lev, groupdict):
    for d in groupdict:
        print("  " * lev + str(d))
        printGroups(lev + 1, groupdict[d])
printGroups(0, g_groupdict)

('General setup', 576, 6747)
  ('After-Start: General setup', 596, 1482)
  ('IRQ subsystem', 1482, 2028)
  ('After-End: IRQ subsystem', 2028, 2377)
  ('Timers subsystem', 2377, 2593)
  ('After-End: Timers subsystem', 2593, 2666)
  ('BPF subsystem', 2666, 2908)
  ('After-End: BPF subsystem', 2908, 3018)
  ('CPU/Task time and stats accounting', 3018, 3417)
  ('After-End: CPU/Task time and stats accounting', 3417, 3442)
  ('RCU Subsystem', 3442, 3697)
  ('After-End: RCU Subsystem', 3697, 3928)
  ('Scheduler features', 3928, 4032)
  ('After-End: Scheduler features', 4032, 6202)
  ('Kernel Performance Events And Counters', 6202, 6359)
('After-End: General setup', 6747, 7727)
('Processor type and features', 7727, 12341)
  ('After-Start: Processor type and features', 7761, 9829)
  ('Performance monitoring', 9829, 10064)
('After-End: Processor type and features', 12341, 12446)
('Power management and ACPI options', 12446, 16179)
  ('After-Start: Power management and ACPI options', 12486, 15064)

In [84]:
cur = 0
for c in g_groupdict:
    if cur > 0:
        print(config[cur:c[1]])
    cur = c[2] + 1

CONFIG_64BIT=y
CONFIG_X86_64=y
CONFIG_X86=y
CONFIG_INSTRUCTION_DECODER=y
CONFIG_OUTPUT_FORMAT="elf64-x86-64"
CONFIG_LOCKDEP_SUPPORT=y
CONFIG_STACKTRACE_SUPPORT=y
CONFIG_MMU=y
CONFIG_ARCH_MMAP_RND_BITS_MIN=28
CONFIG_ARCH_MMAP_RND_BITS_MAX=32
CONFIG_ARCH_MMAP_RND_COMPAT_BITS_MIN=8
CONFIG_ARCH_MMAP_RND_COMPAT_BITS_MAX=16
CONFIG_GENERIC_ISA_DMA=y
CONFIG_GENERIC_BUG=y
CONFIG_GENERIC_BUG_RELATIVE_POINTERS=y
CONFIG_ARCH_MAY_HAVE_PC_FDC=y
CONFIG_GENERIC_CALIBRATE_DELAY=y
CONFIG_ARCH_HAS_CPU_RELAX=y
CONFIG_ARCH_HAS_FILTER_PGPROT=y
CONFIG_HAVE_SETUP_PER_CPU_AREA=y
CONFIG_NEED_PER_CPU_EMBED_FIRST_CHUNK=y
CONFIG_NEED_PER_CPU_PAGE_FIRST_CHUNK=y
CONFIG_ARCH_HIBERNATION_POSSIBLE=y
CONFIG_ARCH_NR_GPIO=1024
CONFIG_ARCH_SUSPEND_POSSIBLE=y
CONFIG_ARCH_WANT_GENERAL_HUGETLB=y
CONFIG_AUDIT_ARCH=y
CONFIG_HAVE_INTEL_TXT=y
CONFIG_X86_64_SMP=y
CONFIG_ARCH_SUPPORTS_UPROBES=y
CONFIG_FIX_EARLYCON_MEM=y
CONFIG_DYNAMIC_PHYSICAL_MASK=y
CONFIG_PGTABLE_LEVELS=5
CONFIG_CC_HAS_SANE_STACKPROTECTOR=y


CONFIG_ARCH_HAS_ADD_